In [1]:
# import python libraries
import pandas as pd
import numpy as np

In [4]:
# importing the CSV file which i get from SQL query (which contains the All free users and their information about online conversion)
table1 = pd.read_csv("Free_Users_And_Their_Online_Conversion.csv")
table1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764401 entries, 0 to 764400
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Category  743423 non-null  object 
 1   mobile    764397 non-null  float64
 2   status1   764401 non-null  object 
 3   status2   31194 non-null   object 
 4   F_Date    764401 non-null  object 
 5   P_Date    31194 non-null   object 
 6   Day_Diff  31194 non-null   float64
dtypes: float64(2), object(5)
memory usage: 40.8+ MB


In [3]:
# importing CSV file which contains all the paid users from offline mode
table2=pd.read_csv("Offline_Paid_Users.csv")
table2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19425 entries, 0 to 19424
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   contact         19425 non-null  int64  
 1   guardianmobile  19425 non-null  int64  
 2   batch_id        19425 non-null  int64  
 3   batch           19425 non-null  object 
 4   category        19425 non-null  object 
 5   branch          19425 non-null  object 
 6   Joining_Date    19425 non-null  object 
 7   Student_Type    19425 non-null  object 
 8   std_type        19425 non-null  object 
 9   amount          19425 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 1.5+ MB


In [11]:
# Joining table1 and table2
table3 = pd.merge(table1, table2, left_on="mobile", right_on="contact", how="left")
table3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764836 entries, 0 to 764835
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Category        743829 non-null  object 
 1   mobile          764832 non-null  float64
 2   status1         764836 non-null  object 
 3   status2         31206 non-null   object 
 4   F_Date          764836 non-null  object 
 5   P_Date          31206 non-null   object 
 6   Day_Diff        31206 non-null   float64
 7   contact         8923 non-null    float64
 8   guardianmobile  8923 non-null    float64
 9   batch_id        8923 non-null    float64
 10  batch           8923 non-null    object 
 11  category        8923 non-null    object 
 12  branch          8923 non-null    object 
 13  Joining_Date    8923 non-null    object 
 14  Student_Type    8923 non-null    object 
 15  std_type        8923 non-null    object 
 16  amount          8923 non-null    float64
dtypes: float64

In [10]:
# Creating Three new columns Date_Diff, Match and F_Date_Month
table3['F_Date'] = pd.to_datetime(table3['F_Date'])
table3['Joining_Date'] = pd.to_datetime(table3['Joining_Date'])
table3['Date_Diff'] = (table3['Joining_Date'] - table3['F_Date']).dt.days
table3['Match'] = np.where(table3['Category'] == table3['category'], 1, 0)
table3['F_Date_Month'] = table3['F_Date'].dt.to_period('M')
table3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764836 entries, 0 to 764835
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Category        743829 non-null  object        
 1   mobile          764832 non-null  float64       
 2   status1         764836 non-null  object        
 3   status2         31206 non-null   object        
 4   F_Date          764836 non-null  datetime64[ns]
 5   P_Date          31206 non-null   object        
 6   Day_Diff        31206 non-null   float64       
 7   contact         8923 non-null    float64       
 8   guardianmobile  8923 non-null    float64       
 9   batch_id        8923 non-null    float64       
 10  batch           8923 non-null    object        
 11  category        8923 non-null    object        
 12  branch          8923 non-null    object        
 13  Joining_Date    8923 non-null    datetime64[ns]
 14  Student_Type    8923 non-null    obj

In [9]:
# Calculating Category wise paid conversion in both online and offline mode Month on month
result = table3.groupby(['Category', 'F_Date_Month']).agg(
    Total_Free_User=('mobile', 'count'),
    Converted_Online_Paid_Users=('mobile', lambda x: (table3.loc[x.index, 'Day_Diff'] >= 0).sum()),
    Converted_Offline_Paid_Users=('mobile', lambda x: ((table3.loc[x.index, 'Date_Diff'] >= 0) & (table3.loc[x.index, 'Match'] == 1)).sum())
).reset_index()
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype    
---  ------                        --------------  -----    
 0   Category                      169 non-null    object   
 1   F_Date_Month                  169 non-null    period[M]
 2   Total_Free_User               169 non-null    int64    
 3   Converted_Online_Paid_Users   169 non-null    int64    
 4   Converted_Offline_Paid_Users  169 non-null    int64    
dtypes: int64(3), object(1), period[M](1)
memory usage: 6.7+ KB
